DSCI Final Project - Baseball Statistics

MLB Stats API: <br>
https://pypi.org/project/MLB-StatsAPI/ <br>
Wiki/Documentation: https://github.com/toddrob99/MLB-StatsAPI/wiki <br>
Usage: run 'pip install MLB-StatsAPI' <br><br>
Datasets:<br>
SFBB-Player-ID-Map.csv: from https://www.smartfantasybaseball.com/tools/ <br>
expected_stats.csv: from https://baseballsavant.mlb.com/leaderboard/expected_statistics?type=batter&year=2015&position=&team=&min=q <br>
Batting.csv: from https://www.kaggle.com/datasets/open-source-sports/baseball-databank?select=Batting.csv

This file, written by Jonathan, deals with expected statistics, as well as comparison with up-to-date statistics, gotten from calls to the MLB Stats API (Question 2/2)


In [36]:
# Setups
import pandas as pd
import plotly.express as px
import os
import statsapi as api
#import logging

In [37]:
data_location = "datasets/"
exp_stats_loc = "exp_stats/"

exp_stats = pd.read_csv(data_location + exp_stats_loc + "expected_stats_2015.csv")
batting_names = pd.read_csv(data_location + 'SFBB-Player-ID-Map.csv')
batting = pd.read_csv(data_location + 'Batting.csv')

In [38]:
#renames columns of exp_stats and makes it easier to view
all_exp_df = []
for year in range(2015, 2024):
    file_name = os.path.join(data_location, exp_stats_loc, 'expected_stats_' + str(year) + '.csv')
    data = pd.read_csv(file_name)
    all_exp_df.append(data)
exp_stats = pd.concat(all_exp_df, ignore_index=True)
exp_stats = exp_stats.rename(columns={'last_name':'Name',
                                      ' first_name':'to_drop',
                                      'player_id':'ID',
                                      'year':'Year',
                                      'pa':'Plate Appearances',
                                      'bip':'Balls In Play',
                                      'ba':'BA',
                                      'est_ba':'xBA',
                                      'est_ba_minus_ba_diff':'xBA - BA Diff',
                                      'slg':'Slg',
                                      'est_slg':'xSlg',
                                      'est_slg_minus_slg_diff':'xSlg - Slg Diff',
                                      'woba':'WOBA',
                                      'est_woba':'xWOBA',
                                      'est_woba_minus_woba_diff':'xWOBA - WOBA Diff'})
exp_stats['Name'] = (exp_stats['to_drop'] + ' ' + exp_stats['Name']).astype('str')
exp_stats = exp_stats.drop(columns =['to_drop'])
#exp_stats = exp_stats.set_index('Name')
exp_stats = exp_stats.replace(r"^ +| +$", r"", regex=True)

In [39]:
#renames columns of batting and makes it easier to view
#display(batting)
batting = batting[batting.yearID >= 2010]
batting = batting.drop(columns=['stint', 'lgID'])
batting = batting.rename(columns = {'playerID':'Player ID',
                                    'yearID':'Year',
                                    'teamID':'Team',
                                    'G':'Games'})
#batting = batting.set_index('Player ID')
display(batting)

,Player ID,Year,Team,Games,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
92849,aardsda01,2010,SEA,53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92850,abadfe01,2010,HOU,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
92851,abreubo01,2010,LAA,154,573.0,88.0,146.0,41.0,1.0,20.0,78.0,24.0,10.0,87.0,132.0,3.0,2.0,0.0,5.0,13.0
92852,abreuto01,2010,ARI,81,193.0,16.0,45.0,11.0,1.0,1.0,13.0,2.0,1.0,4.0,47.0,0.0,0.0,0.0,4.0,8.0
92853,accarje01,2010,TOR,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101327,zitoba01,2015,OAK,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101328,zobribe01,2015,OAK,67,235.0,39.0,63.0,20.0,2.0,6.0,33.0,1.0,1.0,33.0,26.0,2.0,0.0,0.0,3.0,5.0
101329,zobribe01,2015,KCA,59,232.0,37.0,66.0,16.0,1.0,7.0,23.0,2.0,3.0,29.0,30.0,1.0,1.0,0.0,2.0,3.0
101330,zuninmi01,2015,SEA,112,350.0,28.0,61.0,11.0,0.0,11.0,28.0,0.0,1.0,21.0,132.0,0.0,5.0,8.0,2.0,6.0


In [40]:
batting_final = pd.merge(batting_names[['IDPLAYER','PLAYERNAME', 'BATS']], batting,  right_on='Player ID', left_on='IDPLAYER')
batting_final = batting_final.rename(columns = {'PLAYERNAME':'Name'})
#batting_final = batting_final.set_index('Name')

In [41]:
display(batting_final)
display(exp_stats)

,IDPLAYER,Name,BATS,Player ID,Year,Team,Games,AB,R,H,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,aardsda01,David Aardsma,R,aardsda01,2010,SEA,53,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,aardsda01,David Aardsma,R,aardsda01,2012,NYA,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,aardsda01,David Aardsma,R,aardsda01,2013,NYN,43,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,aardsda01,David Aardsma,R,aardsda01,2015,ATL,33,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,abadfe01,Fernando Abad,L,abadfe01,2010,HOU,22,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6830,zobribe01,Ben Zobrist,B,zobribe01,2015,KCA,59,232.0,37.0,66.0,...,23.0,2.0,3.0,29.0,30.0,1.0,1.0,0.0,2.0,3.0
6831,zuninmi01,Mike Zunino,R,zuninmi01,2013,SEA,52,173.0,22.0,37.0,...,14.0,1.0,0.0,16.0,49.0,0.0,3.0,0.0,1.0,5.0
6832,zuninmi01,Mike Zunino,R,zuninmi01,2014,SEA,131,438.0,51.0,87.0,...,60.0,0.0,3.0,17.0,158.0,1.0,17.0,0.0,4.0,12.0
6833,zuninmi01,Mike Zunino,R,zuninmi01,2015,SEA,112,350.0,28.0,61.0,...,28.0,0.0,1.0,21.0,132.0,0.0,5.0,8.0,2.0,6.0


,Name,ID,Year,Plate Appearances,Balls In Play,BA,xBA,xBA - BA Diff,Slg,xSlg,xSlg - Slg Diff,WOBA,xWOBA,xWOBA - WOBA Diff
0,Marcus Semien,543760,2022,724,547,0.248,0.243,0.005,0.429,0.394,0.035,0.317,0.306,0.011
1,Freddie Freeman,518692,2022,708,517,0.325,0.313,0.012,0.511,0.538,-0.027,0.393,0.403,-0.010
2,Trea Turner,607208,2022,708,527,0.298,0.276,0.022,0.466,0.432,0.034,0.350,0.335,0.015
3,Vladimir Guerrero Jr.,665489,2022,706,526,0.274,0.276,-0.002,0.480,0.462,0.018,0.351,0.348,0.003
4,Francisco Lindor,596019,2022,706,504,0.270,0.254,0.016,0.449,0.427,0.022,0.342,0.331,0.011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258,Zach McKinstry,656716,2023,64,44,0.271,0.286,-0.015,0.458,0.508,-0.050,0.336,0.360,-0.024
2259,Sam Hilliard,656541,2023,64,30,0.276,0.224,0.052,0.500,0.411,0.089,0.365,0.309,0.056
2260,David Hensley,682073,2023,63,33,0.143,0.170,-0.027,0.161,0.223,-0.062,0.197,0.230,-0.033
2261,Eduardo Escobar,500871,2023,63,46,0.172,0.223,-0.051,0.397,0.348,0.049,0.256,0.269,-0.013


In [42]:
#an unfortunately messy function that parses data returned by API calls, and converts it into a DataFrame
def parse_player_api_data(data):
    lines = data.split('\n')
    line = lines[0]
    name_start = 0
    name_end = line.find('"') - 1
    last_name_start = line.find(' ') + 1
    last_name_end = line.find(',', last_name_start)
    name = line[name_start:name_end+1]
    first_word_end = line.find(' ')
    if first_word_end == -1:
        name = line
    else:
        name = line[:first_word_end]
    last_name = line[last_name_start:last_name_end]
    nickname_start = last_name.find('"')
    nickname_end = last_name.find('"', nickname_start + 1)
    if nickname_start != -1 and nickname_end != -1:
        last_name = last_name[:nickname_start] + last_name[nickname_end+1:]\

            
    career_hitting = {}
    for line in lines[3:]:
        if line:
            key, value = line.split(': ')
            career_hitting[key] = value
    df = pd.DataFrame(career_hitting, index=[0])
    df['Name'] = name
    df['LastName'] = last_name
    df = df[['Name','LastName'] + list(career_hitting.keys())]
    
    numeric_cols = ['gamesPlayed', 'groundOuts', 'runs', 'doubles', 'triples', 'homeRuns', 'strikeOuts', 'baseOnBalls',
                    'intentionalWalks', 'hits', 'hitByPitch', 'atBats', 'caughtStealing', 'stolenBases',
                    'groundIntoDoublePlay', 'numberOfPitches', 'plateAppearances', 'totalBases', 'rbi', 'leftOnBase',
                    'sacBunts', 'sacFlies']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric)
    
    return df

In [43]:


df_for_api_call = exp_stats[exp_stats.Year == 2023].drop_duplicates(subset=['Name'], keep=False)
player_season = pd.DataFrame()
player_career = pd.DataFrame()

for playerID in df_for_api_call['ID']:
    #f.write(api.player_stats(playerID, group='hitting', type='season'))
    season_data = parse_player_api_data(api.player_stats(playerID, group='hitting', type='season'))
    career_data = parse_player_api_data(api.player_stats(playerID, group='hitting', type='career'))
    player_season = pd.concat([player_season,season_data])
    player_career = pd.concat([player_career, career_data])

#display(player_season)
#display(player_career)

KeyboardInterrupt: 

In [ ]:
#Cleaning dataframes returned from API
player_season['FullName'] = player_season['Name'] + ' ' + player_season['LastName']
player_career['FullName'] = player_career['Name'] + ' ' + player_career['LastName']
player_season = player_season.drop(columns=['Name','LastName'])
player_career = player_career.drop(columns=['Name','LastName'])

display(player_season)
display(player_career)

,gamesPlayed,groundOuts,airOuts,runs,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,...,totalBases,rbi,leftOnBase,sacBunts,sacFlies,babip,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun,FullName
0,36,31,43,27,10,0,5,28,17,2,...,66,14,52,0,2,.327,0.72,0,28.20,Freddie Freeman
0,35,40,35,27,10,1,5,20,13,1,...,69,18,47,0,0,.330,1.14,0,27.40,Alex Verdugo
0,35,40,46,18,5,1,1,16,20,2,...,48,12,53,0,2,.301,0.87,0,138.00,Steven Kwan
0,35,49,40,25,6,1,2,17,9,0,...,60,19,43,0,1,.331,1.23,0,75.50,Nico Hoerner
0,35,28,42,21,6,2,4,39,9,2,...,61,9,54,0,0,.333,0.67,0,37.00,Trea Turner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,28,17,20,12,5,0,2,19,10,0,...,30,6,26,1,1,.321,0.85,0,36.50,Zach McKinstry
0,25,8,10,11,5,0,3,29,6,0,...,31,6,23,0,0,.438,0.80,0,21.33,Sam Hilliard
0,20,18,11,9,1,0,0,25,8,0,...,9,2,37,0,0,.216,1.64,0,-.--,David Hensley
0,20,21,19,8,2,1,3,15,4,0,...,23,9,45,1,1,.152,1.11,0,21.00,Eduardo Escobar


,gamesPlayed,groundOuts,airOuts,runs,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,...,totalBases,rbi,leftOnBase,sacBunts,sacFlies,babip,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun,FullName
0,1760,1500,1684,1113,424,27,297,1442,877,116,...,3313,1055,2444,0,50,.341,0.89,0,21.95,Freddie Freeman
0,544,601,470,281,125,6,49,314,159,11,...,836,219,790,1,18,.320,1.28,0,39.14,Alex Verdugo
0,182,189,238,107,30,8,7,76,82,4,...,273,64,270,2,6,.319,0.79,0,100.14,Steven Kwan
0,282,297,271,130,43,7,15,134,69,8,...,374,120,346,0,7,.312,1.10,1,64.47,Nico Hoerner
0,884,894,913,607,197,38,128,712,277,10,...,1729,443,1189,2,17,.343,0.98,2,27.91,Trea Turner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,149,100,99,53,21,3,14,124,36,1,...,157,49,173,4,3,.275,1.01,0,28.86,Zach McKinstry
0,239,135,125,95,24,7,32,238,70,3,...,272,77,271,0,2,.290,1.08,0,19.81,Sam Hilliard
0,36,27,15,16,3,1,1,31,13,0,...,26,7,50,0,0,.288,1.80,0,91.00,David Hensley
0,1284,995,1416,569,243,42,161,980,337,25,...,1940,614,1996,14,41,.288,0.70,0,27.74,Eduardo Escobar


In [ ]:
combined_data = pd.concat([player_season.assign(DataType='Season'),
                           player_career.assign(DataType='Career')])
combined_data = combined_data.sort_values('avg', ascending=True)
fig = px.bar(combined_data, x='avg', y='FullName', color='DataType',
             title='Comparison of Batting Average between Season and Career',
             orientation='v')
fig.update_layout(yaxis=dict(title=''))
fig.show()

In [ ]:
combined_data = pd.concat([player_season.assign(DataType='Season'),
                           player_career.assign(DataType='Career')])
fig = px.bar(combined_data, x='slg', y='FullName', color='DataType',
             title='Comparison of Slugging between Season and Career',
             orientation='v')
fig.update_layout(yaxis=dict(title=''))
fig.show()

In [ ]:
combined_data = pd.concat([player_season.assign(DataType='Season'),
                           player_career.assign(DataType='Career')])
fig = px.bar(combined_data, x='obp', y='FullName', color='DataType',
             title='Comparison of OBP between Season and Career',
             orientation='v')
fig.update_layout(yaxis=dict(title=''))
fig.show()

In [ ]:
combined_data = pd.concat([player_season.assign(DataType='Season'),
                           player_career.assign(DataType='Career')])
fig = px.bar(combined_data, x='groundOutsToAirouts', y='FullName', color='DataType',
             title='Comparison of Groud-to-Air Outs between Season and Career',
             orientation='v')
fig.update_layout(yaxis=dict(title=''))
fig.show()

In [ ]:
#display(exp_stats)
yearly_avg = exp_stats.groupby('Year').mean().reset_index()
fig = px.line(yearly_avg, x='Year', y=['BA', 'Slg', 'WOBA'], 
              title='BA, Slg, and WOBA changes over time')
fig.show()

C:\Users\HOMANJP20\AppData\Local\Temp\ipykernel_7604\2057035568.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

